In [5]:
# Copyright 2019 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the \"License\");
# you may not use this file except in compliance with the License.\n",
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an \"AS IS\" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

from __future__ import print_function

import os
import subprocess
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets
from torchvision import transforms

MODEL_FILE_NAME = 'torch.model'


class Net(nn.Module):

  def __init__(self):
    super(Net, self).__init__()
    self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
    self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
    self.conv2_drop = nn.Dropout2d()
    self.fc1 = nn.Linear(320, 50)
    self.fc2 = nn.Linear(50, 10)

  def forward(self, x):
    x = F.relu(F.max_pool2d(self.conv1(x), 2))
    x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
    x = x.view(-1, 320)
    x = F.relu(self.fc1(x))
    x = F.dropout(x, training=self.training)
    x = self.fc2(x)
    return F.log_softmax(x, dim=1)


def train(args, model, device, train_loader, optimizer, epoch):
  model.train()
  for batch_idx, (data, target) in enumerate(train_loader):
    data, target = data.to(device), target.to(device)
    optimizer.zero_grad()
    output = model(data)
    loss = F.nll_loss(output, target)
    loss.backward()
    optimizer.step()
    if batch_idx % 100 == 0:
      print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
          epoch, batch_idx * len(data), len(train_loader.dataset),
          100. * batch_idx / len(train_loader), loss.item()))


def test(args, model, device, test_loader, epoch):
  model.eval()
  test_loss = 0
  correct = 0
  with torch.no_grad():
    for data, target in test_loader:
      data, target = data.to(device), target.to(device)
      output = model(data)
      test_loss += F.nll_loss(
          output, target, size_average=False).item()  # sum up batch loss
      pred = output.max(
          1, keepdim=True)[1]  # get the index of the max log-probability
      correct += pred.eq(target.view_as(pred)).sum().item()

  test_loss /= len(test_loader.dataset)
  print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
      test_loss, correct, len(test_loader.dataset),
      100. * correct / len(test_loader.dataset)))


def main():
  # Training settings
  args = {"batch_size": 64,
          "test_batch_size": 1000,
          "epochs": 1,
          "lr": 0.01,
          "momentum": 0.5,
          "no_cuda": True,
          "seed": 42,
          "log_interval": 10,
          "model_dir": None}
    
  use_cuda = not args['no_cuda'] and torch.cuda.is_available()

  torch.manual_seed(42)

  device = torch.device('cuda' if use_cuda else 'cpu')

  kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
  train_loader = torch.utils.data.DataLoader(
    datasets.MNIST(
      './data',
      train=True,
      download=True,
      transform=transforms.Compose([
        transforms.ToTensor(),
        # Normalize a tensor image with mean and standard deviation
        transforms.Normalize(mean=(0.1307,), std=(0.3081,))
      ])),
      batch_size=64,
      shuffle=True,
      **kwargs)
  test_loader = torch.utils.data.DataLoader(
    datasets.MNIST(
      './data',
      train=False,
      transform=transforms.Compose([
        transforms.ToTensor(),
        # Normalize a tensor image with mean and standard deviation
        transforms.Normalize(mean=(0.1307,), std=(0.3081,))
      ])),
      batch_size=64,
      shuffle=True,
      **kwargs)

  model = Net().to(device)
  optimizer = optim.SGD(model.parameters(), lr=args['lr'], momentum=args['momentum'])

  for epoch in range(1, args['epochs'] + 1):
    train(args, model, device, train_loader, optimizer, epoch)
    test(args, model, device, test_loader, epoch)

  if args['model_dir']:
    tmp_model_file = os.path.join('/tmp', MODEL_FILE_NAME)
    torch.save(model.state_dict(), tmp_model_file)
    subprocess.check_call([
        'gsutil', 'cp', tmp_model_file,
        os.path.join(args['model_dir'], MODEL_FILE_NAME)])


main()


Train Epoch: 1 [0/60000 (0%)]	Loss: 2.310367
Train Epoch: 1 [6400/60000 (11%)]	Loss: 2.180723
Train Epoch: 1 [12800/60000 (21%)]	Loss: 1.560432
Train Epoch: 1 [19200/60000 (32%)]	Loss: 1.093575
Train Epoch: 1 [25600/60000 (43%)]	Loss: 0.777248
Train Epoch: 1 [32000/60000 (53%)]	Loss: 0.540999
Train Epoch: 1 [38400/60000 (64%)]	Loss: 0.460816
Train Epoch: 1 [44800/60000 (75%)]	Loss: 0.493297
Train Epoch: 1 [51200/60000 (85%)]	Loss: 0.661505
Train Epoch: 1 [57600/60000 (96%)]	Loss: 0.445722


/opt/conda/lib/python3.7/site-packages/torch/nn/_reduction.py:43: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))



Test set: Average loss: 0.2043, Accuracy: 9398/10000 (94%)

